# Final Script for flags
Joins flags_auto, osu world and flag_additions


In [1]:
import requests
import json
import copy


In [2]:
def fetchData(api_url):
    try:
        response = requests.get(api_url)

        if response.status_code == 200:

            data = response.json()
            return data
        else:
            print(f"Request failed with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)


## Clean flags auto leaving only the regions in osu world


In [3]:
flagsPathAuto = "./out/flags_auto.json"

with open(flagsPathAuto, "r", encoding="utf-8") as json_file:
    flagsAuto = json.load(json_file)


Add countries from osuworld

In [4]:
countries: dict = fetchData("https://osuworld.octo.moe/locales/en/countries.json")

In [5]:
for countryCode, country in countries.items():
    if countryCode not in flagsAuto:
        flagsAuto[countryCode] = {}
        flagsAuto[countryCode]['name'] = country
        flagsAuto[countryCode]['regions'] = {}
        print(f"Country {country} not found in flagsAuto.json")

Country Afghanistan not found in flagsAuto.json
Country Armenia not found in flagsAuto.json
Country Angola not found in flagsAuto.json
Country Barbados not found in flagsAuto.json
Country Bangladesh not found in flagsAuto.json
Country Burkina Faso not found in flagsAuto.json
Country Bulgaria not found in flagsAuto.json
Country Brunei not found in flagsAuto.json
Country Dominican Republic not found in flagsAuto.json
Country Algeria not found in flagsAuto.json
Country Greece not found in flagsAuto.json
Country Israel not found in flagsAuto.json
Country India not found in flagsAuto.json
Country Iceland not found in flagsAuto.json
Country Jamaica not found in flagsAuto.json
Country Jordan not found in flagsAuto.json
Country Cambodia not found in flagsAuto.json
Country Kazakhstan not found in flagsAuto.json
Country Laos not found in flagsAuto.json
Country Lebanon not found in flagsAuto.json
Country Luxembourg not found in flagsAuto.json
Country Morocco not found in flagsAuto.json
Country No

add regions from osu world

In [6]:
regions: dict = fetchData("https://osuworld.octo.moe/locales/en/regions.json")


In [7]:
flags: dict = copy.deepcopy(flagsAuto)

for countryCode, country in flagsAuto.items():
    osuWorldRegions: dict = regions.get(countryCode)
    if osuWorldRegions is None:
        flags.pop(countryCode, None)
        continue
    print(f"Country {country['name']} found in regions.json")
    for regionCode, region in country['regions'].items():
        osuWorldRegionKeys = osuWorldRegions.keys()
        if regionCode in osuWorldRegionKeys:
            region['name'] = osuWorldRegions[regionCode]
        else:
            flags[countryCode]['regions'].pop(regionCode, None)

    for osuWorldRegion in osuWorldRegions:
        if osuWorldRegion not in country['regions']:
            flags[countryCode]['regions'][osuWorldRegion] = {
                "name": osuWorldRegions[osuWorldRegion],
                "nativeName": "",
                "flag": ""
            }


Country Albania found in regions.json
Country Andorra found in regions.json
Country Argentina found in regions.json
Country Australia found in regions.json
Country Austria found in regions.json
Country Bahrain found in regions.json
Country Belarus found in regions.json
Country Belgium found in regions.json
Country Bolivia found in regions.json
Country Bosnia and Herzegovina found in regions.json
Country Brazil found in regions.json
Country Canada found in regions.json
Country Chile found in regions.json
Country China found in regions.json
Country Colombia found in regions.json
Country Costa Rica found in regions.json
Country Croatia found in regions.json
Country Czech Republic found in regions.json
Country Denmark found in regions.json
Country Ecuador found in regions.json
Country Egypt found in regions.json
Country El Salvador found in regions.json
Country Estonia found in regions.json
Country Finland found in regions.json
Country France found in regions.json
Country Georgia found in 

Save

In [8]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


Reload data

In [9]:
flagsPath = "./out/flags.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)


## Add Manual changes to the automatic flags

In [10]:
flagsAdditionsPath = "./in/flags_additions.json"

with open(flagsAdditionsPath, "r", encoding="utf-8") as json_file:
    flagsAdditions = json.load(json_file)


In [11]:
import collections.abc

def update(d, u):
    for k, v in u.items():
        if isinstance(v, collections.abc.Mapping):
            d[k] = update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

In [12]:
update(flags, flagsAdditions)

{'AL': {'name': 'Albania',
  'nativeName': 'Shqipëria',
  'regions': {'AL-01': {'name': 'Berat County',
    'nativeName': 'Qarku i Beratit',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/4/49/Flag_of_Berat.svg'},
   'AL-02': {'name': 'Durrës County',
    'nativeName': 'Qarku i Durrësit',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/e/e4/Flag_of_Durrës.svg'},
   'AL-03': {'name': 'Elbasan County',
    'nativeName': 'Qarku i Elbasanit',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/4/4d/Flag_of_Elbasan.svg'},
   'AL-06': {'name': 'Korçë County',
    'nativeName': 'Qarku i Korçës',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/2/2b/Flag_of_Korçë.svg'},
   'AL-07': {'name': 'Kukës County',
    'nativeName': 'Qarku i Kukësit',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/7/7e/Flag_of_Kukës.svg'},
   'AL-11': {'name': 'Tirana County',
    'nativeName': 'Qarku i Tiranës',
    'flag': 'https://upload.wikimedia.org/wikipedi

Save again


In [13]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


## Fill native names

Using Open Street Maps API: https://nominatim.openstreetmap.org/ui/search.html


In [14]:
urlSearch = "https://nominatim.openstreetmap.org/search.php?q=$1&countrycodes=$2&format=jsonv2"


def getSearchURL(regionName, countryCode):
    return urlSearch.replace("$1", regionName).replace("$2", countryCode)


urlReverseId = "https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=$1&format=json"


def getReverseIdURL(id):
    return urlReverseId.replace("$1", id)


In [15]:
def addResult(region, matchData):
    localName = matchData['name']
    if not localName:
        return False

    region["nativeName"] = localName
    print(localName)
    return True


def handleSearch(region, data):
    if data:
        it = (d for d in data if d.get("type") == "administrative")

        matchData = next(it, None)

        while matchData:
            suceed = addResult(region, matchData)
            if not suceed:
                matchData = next(it, None)
                if matchData is not None:
                    print("Trying next match")
            else:
                matchData = None
                return True

    else:
        print("No match data")
    return False


country: dict
for countryCode, country in flags.items():
    print(countryCode)

    nativeName = country.get('nativeName')
    if nativeName is None or nativeName == "":
        api_url = getSearchURL(country['name'], countryCode)
        data = fetchData(api_url)
        searchHandleResult = handleSearch(country, data)

        # Reorder regions to appear after nativeName
        regions = country['regions']
        del country['regions']
        country['regions'] = regions

    for regionCode, region in country['regions'].items():
        if region['nativeName'] != "":
            continue

        api_url = getSearchURL(regionCode, countryCode)
        data = fetchData(api_url)
        searchHandleResult = handleSearch(region, data)
        if not searchHandleResult:
            name = region['name']
            api_url = getSearchURL(name, countryCode)
            data = fetchData(api_url)
            searchHandleResult = handleSearch(region, data)


AL
AD
AR


Salta
Buenos Aires
San Luis
La Rioja
Santiago del Estero
Chaco
San Juan
Catamarca
La Pampa
Mendoza
Misiones
Formosa
Santa Fe
Tucumán
Corrientes
Córdoba
Jujuy
AU
AT
BH
BY
BE
BO
BA
BR
CA
CL
Región de Arica y Parinacota
CN
CO
Amazonas
Antioquia
Arauca
Atlántico
Bolívar
Boyacá
Caldas
Caquetá
Casanare
Cauca
Cesar
Chocó
Córdoba
Cundinamarca
Guainía
Guaviare
Huila
La Guajira
Magdalena
Meta
Nariño
Norte de Santander
Putumayo
Quindío
Risaralda
Santander
Archipelago of Saint Andrew, Providence and Saint Catherine
Sucre
Tolima
Valle del Cauca
Vaupés
Vichada
CR
HR
CZ
Jihomoravský kraj
Karlovarský kraj
Královéhradecký kraj
Liberecký kraj
Moravskoslezský kraj
Olomoucký kraj
Plzeňský kraj
Středočeský kraj
Ústecký kraj
DK
EC
EG
SV
EE
Harju maakond
Hiiu maakond
Ida-Viru maakond
Jõgeva maakond
Järva maakond
Lääne maakond
Lääne-Viru maakond
Põlva maakond
Pärnu maakond
Rapla maakond
Saare maakond
Tartu maakond
Valga maakond
Viljandi maakond
Võru maakond
FI
Etelä-Karjala
Kymenlaakso
Lappi
Pirkanmaa
Österbo

In [16]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


## Reload data


In [17]:
flagsPath = "./out/flags.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)


In [18]:
[region['name'] for region in flags["ES"]["regions"].values()]


['Andalusia',
 'Aragon',
 'Asturias',
 'Basque Country',
 'Canary Islands',
 'Cantabria',
 'Castilla–La Mancha',
 'Castile and León',
 'Catalonia',
 'Extremadura',
 'Galicia',
 'Community of Madrid',
 'Region of Murcia',
 'Navarre',
 'Valencian Community',
 'Ceuta',
 'Melilla',
 'Balearic Islands',
 'La Rioja']

Countries missing regions

In [19]:
printJsons = False

for countryCode, country in flags.items():
    regions = country["regions"]
    flagsFilter = [region['flag'] for region in regions.values()]
    containsEmptyFlag = any(flag == "" for flag in flagsFilter)
    if containsEmptyFlag:
        print("-", country["name"])
        if printJsons:
            print(json.dumps(country, indent=4, ensure_ascii=False))

- Georgia
- Germany
- Honduras
- Iraq
- Russia
- Uzbekistan
- Afghanistan
- Armenia
- Angola
- Bangladesh
- Burkina Faso
- Brunei
- Dominican Republic
- Algeria
- Jordan
- Qatar
- Slovenia
- Tunisia
